### Data Preparation

In [1]:
%matplotlib inline
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import matplotlib as plt
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("./Data/charity_data.csv")

# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN', 'NAME'], axis=1)

In [2]:
# Look at APPLICATION_TYPE value counts for binning
at_vc = application_df.APPLICATION_TYPE.value_counts()

# Determine which values to replace if counts are less than ...?
replace_application = list(at_vc[at_vc < 1000].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: APPLICATION_TYPE, dtype: int64

In [3]:
# Look at CLASSIFICATION value counts for binning
class_vc = application_df.CLASSIFICATION.value_counts()

# Determine which values to replace if counts are less than ..?
replace_class = list(class_vc[class_vc < 5000].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
Other    10899
C2000     6074
Name: CLASSIFICATION, dtype: int64

In [4]:
# Generate our categorical variable lists
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()

# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)

# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df,left_index=True, right_index=True)
application_df = application_df.drop(application_cat,1)

C:\Users\amlab\anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\amlab\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:15: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  from ipykernel import kernelapp as app


In [5]:
# Split our preprocessed data into our features and target arrays
y = application_df['IS_SUCCESSFUL'].values
X = application_df.drop(['IS_SUCCESSFUL'],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

C:\Users\amlab\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


### D3 Optimaztion

In [6]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE

number_input_features = len(X_train[0])
hidden_nodes_layer1 = 150
hidden_nodes_layer2 = 100
hidden_nodes_layer3 = 80
hidden_nodes_layer4 = 60
hidden_nodes_layer5 = 50
hidden_nodes_layer6 = 20

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="tanh"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Fourth hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="sigmoid"))

# Fifth hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer5, activation="tanh"))

# Sixth hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer6, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 150)               5700      
                                                                 
 dense_1 (Dense)             (None, 100)               15100     
                                                                 
 dense_2 (Dense)             (None, 80)                8080      
                                                                 
 dense_3 (Dense)             (None, 60)                4860      
                                                                 
 dense_4 (Dense)             (None, 50)                3050      
                                                                 
 dense_5 (Dense)             (None, 20)                1020      
                                                                 
 dense_6 (Dense)             (None, 1)                 2

In [7]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [8]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [9]:
# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=1000)

In [10]:
# Train the model
fit_model = nn.fit(X_train, y_train,epochs=200)

Epoch 1/200
804/804 [==============================] - 4s 4ms/step - loss: 0.6933 - accuracy: 0.5237
Epoch 2/200
804/804 [==============================] - 3s 4ms/step - loss: 0.6923 - accuracy: 0.5273
Epoch 3/200
804/804 [==============================] - 3s 4ms/step - loss: 0.6917 - accuracy: 0.5302
Epoch 4/200
804/804 [==============================] - 3s 4ms/step - loss: 0.6918 - accuracy: 0.5316
Epoch 5/200
804/804 [==============================] - 4s 5ms/step - loss: 0.6917 - accuracy: 0.5279
Epoch 6/200
804/804 [==============================] - 4s 5ms/step - loss: 0.6913 - accuracy: 0.5309
Epoch 7/200
804/804 [==============================] - 4s 4ms/step - loss: 0.6913 - accuracy: 0.5321
Epoch 8/200
804/804 [==============================] - 3s 4ms/step - loss: 0.6912 - accuracy: 0.5321
Epoch 9/200
804/804 [==============================] - 4s 5ms/step - loss: 0.6912 - accuracy: 0.5321
Epoch 10/200
804/804 [==============================] - 4s 5ms/step - loss: 0.6912 - accura

804/804 [==============================] - 4s 5ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 82/200
804/804 [==============================] - 3s 4ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 83/200
804/804 [==============================] - 4s 4ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 84/200
804/804 [==============================] - 4s 4ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 85/200
804/804 [==============================] - 4s 5ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 86/200
804/804 [==============================] - 3s 4ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 87/200
804/804 [==============================] - 3s 4ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 88/200
804/804 [==============================] - 3s 4ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 89/200
804/804 [==============================] - 4s 5ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 90/200
804/804 [==============================] - 4s 5ms/step - loss: 0.6911 - accuracy: 

804/804 [==============================] - 4s 5ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 161/200
804/804 [==============================] - 4s 4ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 162/200
804/804 [==============================] - 3s 4ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 163/200
804/804 [==============================] - 3s 4ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 164/200
804/804 [==============================] - 4s 5ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 165/200
804/804 [==============================] - 4s 5ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 166/200
804/804 [==============================] - 4s 4ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 167/200
804/804 [==============================] - 3s 4ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 168/200
804/804 [==============================] - 3s 4ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 169/200
804/804 [==============================] - 4s 5ms/step - loss: 0.6911 - a

In [11]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.6904 - accuracy: 0.5332 - 1s/epoch - 4ms/step
Loss: 0.6903836727142334, Accuracy: 0.5331778526306152


In [12]:
# Export our model to HDF5 file
 nn.save("AlphabetSoupCharity_Optimazation.h5")